<a href="https://colab.research.google.com/github/danielsuassuna04/TransferLearning-keras/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras

2024-09-19 23:27:29.567151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 23:27:29.567332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 23:27:29.667163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 23:27:29.873018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 23:27:31.751105: W tensorflow/compiler/tf2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()


batch_size = 32


def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = (train_dataset
                 .map(lambda x, y: preprocess(x, y))
                 .shuffle(buffer_size=10000)
                 .batch(batch_size)
                 .prefetch(1))
test_dataset = (test_dataset
                .map(lambda x, y: preprocess(x, y))
                .batch(batch_size)
                .prefetch(1))

2024-09-19 23:27:37.389589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-19 23:27:37.748287: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-19 23:27:37.748548: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
base_model = keras.applications.ResNet152(weights="imagenet",include_top=False)
base_model.trainable = False
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs,training=False)
avg = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(avg)
output = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs,outputs=output)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01,weight_decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
early = keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)
history = model.fit(train_dataset,epochs=5,validation_data=test_dataset,callbacks=[early,checkpoint])

Epoch 1/5


2024-09-19 23:28:06.756775: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-09-19 23:28:11.375595: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7b3dfbfab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-19 23:28:11.375648: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2024-09-19 23:28:11.396809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1726788491.531028     545 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - ETA: 0s - loss: 1.7728 - accuracy: 0.8323
Epoch 1: val_loss improved from inf to 1.24871, saving model to best_model.weights.h5
1563/1563 [==============================] - 267s 156ms/step - loss: 1.7728 - accuracy: 0.8323 - val_loss: 1.2487 - val_accuracy: 0.8971
Epoch 2/5
1563/1563 [==============================] - ETA: 0s - loss: 2.0809 - accuracy: 0.8599
Epoch 2: val_loss did not improve from 1.24871
1563/1563 [==============================] - 241s 150ms/step - loss: 2.0809 - accuracy: 0.8599 - val_loss: 1.5233 - val_accuracy: 0.8945
Epoch 3/5
1563/1563 [==============================] - ETA: 0s - loss: 2.1734 - accuracy: 0.8665
Epoch 3: val_loss did not improve from 1.24871
1563/1563 [==============================] - 243s 152ms/step - loss: 2.1734 - accuracy: 0.8665 - val_loss: 1.4692 - val_accuracy: 0.8989
Epoch 4/5
1563/1563 [==============================] - ETA: 0s - loss: 2.2504 - accuracy: 0.8681
Epoch 4: val_loss did not impro

In [ ]:
model.load_weights("best_model.weights.h5")

In [ ]:
model.evaluate(test_dataset)

313/313 [==============================] - 38s 123ms/step - loss: 1.2487 - accuracy: 0.8971


[1.2487115859985352, 0.8970999717712402]

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
early = keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model_2.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)
history = model.fit(train_dataset,epochs=5,validation_data=test_dataset,callbacks=[early,checkpoint])

Epoch 1/5
1563/1563 [==============================] - ETA: 0s - loss: 1.3966 - accuracy: 0.8856
Epoch 1: val_loss improved from inf to 0.90592, saving model to best_model_2.weights.h5
1563/1563 [==============================] - 255s 155ms/step - loss: 1.3966 - accuracy: 0.8856 - val_loss: 0.9059 - val_accuracy: 0.9158
Epoch 2/5
1563/1563 [==============================] - ETA: 0s - loss: 1.2314 - accuracy: 0.8887
Epoch 2: val_loss improved from 0.90592 to 0.79446, saving model to best_model_2.weights.h5
1563/1563 [==============================] - 245s 153ms/step - loss: 1.2314 - accuracy: 0.8887 - val_loss: 0.7945 - val_accuracy: 0.9184
Epoch 3/5
1563/1563 [==============================] - ETA: 0s - loss: 1.1118 - accuracy: 0.8926
Epoch 3: val_loss improved from 0.79446 to 0.75195, saving model to best_model_2.weights.h5
1563/1563 [==============================] - 244s 153ms/step - loss: 1.1118 - accuracy: 0.8926 - val_loss: 0.7519 - val_accuracy: 0.9192
Epoch 4/5
1563/1563 [=====

In [ ]:
model.load_weights("best_model_2.weights.h5")

In [ ]:
model.evaluate(test_dataset)

313/313 [==============================] - 38s 122ms/step - loss: 0.6755 - accuracy: 0.9211


[0.6755410432815552, 0.9211000204086304]